# Inference

In [1]:
import numpy as np
import random
import os
import math
from itertools import product

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler


from glob import glob
import pandas as pd
import cv2
from tqdm.auto import tqdm
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
# from torch.utils.tensorboard import SummaryWriter

from torch.autograd import Variable

import torchvision.models as models
from torchvision import transforms

import albumentations as A
import albumentations.pytorch

from sklearn.metrics import accuracy_score

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")

Using cpu device


In [3]:
import tensorflow as tf
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [37]:
CFG = {
    'IMG_SIZE':256,
    'EPOCHS':50,
    'PATIENCE':10,
    'class':41
}

In [5]:
os.listdir('./data/quasi_drug')

['RegNet_0.0001_rmsprop_CosineAnnealing_example.pth',
 'RegNet_0.001_rmsprop_CosineAnnealing_example.pth',
 'RegNet_1e-05_rmsprop_CosineAnnealing_example.pth']

In [6]:
# Train_path = '/home/lab17/jupyter_home/Data/product_image/Training/'
# Valid_path = '/home/lab17/jupyter_home/Data/product_image/Validation/'
# Test_path = '/home/lab17/jupyter_home/git/test_img/'
# save_graph_path = './result/'
# save_model_path = '/home/lab17/jupyter_home/saved_models/'

In [7]:
Train_path = './data/quasi_drug_dataset'
Test_path = 'C:/jupyter_home/GIT/mulcam3projcet_ai/test_img/quasi-drugs'

# Load Data

### train

In [8]:
def get_train_data(data_dir):
#     img_path_list = []
    label_list = []
    label_name_list = []
    
#     image_path = os.path.join(data_dir, 'dessert')
    image_path = data_dir
    
    for product_name in os.listdir(image_path):
        product_path = os.path.join(image_path, product_name)
        if os.path.isdir(product_path):
#             get image path
#             img_path_list.extend(glob(os.path.join(product_path, '*.jpg')))
#             img_path_list.extend(glob(os.path.join(product_path, '*.png')))
            label = list(product_name[:5])
            name = product_name[6:]
            
            # get label
            label_list.append(''.join(label))
            label_name_list.append(name)
                
#     return img_path_list, label_list
    return label_list, label_name_list

In [9]:
label_list, label_name_list = get_train_data(Train_path)

In [10]:
# encoder
le = preprocessing.LabelEncoder()
targets = le.fit_transform(label_list)
print('--targets\n' , targets)

label_encoder = {key:val for key, val in zip(label_list, targets)}
print(label_encoder)

--targets
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40]
{'10096': 0, '10097': 1, '10106': 2, '10213': 3, '10214': 4, '10215': 5, '10266': 6, '10320': 7, '10321': 8, '15258': 9, '15458': 10, '15664': 11, '15804': 12, '15896': 13, '20196': 14, '20260': 15, '30081': 16, '30163': 17, '30164': 18, '30181': 19, '30289': 20, '40182': 21, '40191': 22, '40303': 23, '45707': 24, '50139': 25, '55734': 26, '66380': 27, '70162': 28, '70163': 29, '70180': 30, '70181': 31, '70182': 32, '80140': 33, '80146': 34, '80147': 35, '80249': 36, '80250': 37, '90174': 38, '90176': 39, '90245': 40}


In [11]:
# decoder
label_decoder = {v: k for k, v in label_encoder.items()}
label_name_decoder = {key : value for key, value in zip(label_list, label_name_list)}

### validation

In [12]:
# def get_valid_data(data_dir):
#     img_valid_list = []
#     label_valid_list = []
    
#     image_path = os.path.join(data_dir, 'dessert')
    
#     for product_name in os.listdir(image_path):
#         product_path = os.path.join(image_path, product_name)
#         if os.path.isdir(product_path):
#             # get image path
#             img_valid_list.extend(glob(os.path.join(product_path, '*.jpg')))
#             img_valid_list.extend(glob(os.path.join(product_path, '*.png')))
#             label = list(product_name[:5])
            
#             # get label
#             label_valid_list.append(''.join(label))
                
#     return img_valid_list, label_valid_list

In [13]:
# def valid_data_blanced(img, label):
#     x = []
#     y = []
    
#     for i in range(CFG['class']):
#         _img = img[(i * 15): ((i + 1) * 15)]
#         _label = label[i]
        
#         for img_product in _img:
#             x.append(img_product)
#             y.append(_label)
            
#     return x, y

In [14]:
# img_valid_list, label_valid_list = get_valid_data(Valid_path)
# x_valid, y_valid = valid_data_blanced(img_valid_list, label_valid_list)
# len(label_valid_list)

In [15]:
# le2 = preprocessing.LabelEncoder()
# targets_y = le2.fit_transform(y_valid)
# targets_y_t = torch.as_tensor(targets_y)
# one_hot_valid_y = F.one_hot(targets_y_t)
# one_hot_valid_y.shape

### test

In [16]:
def get_test_data(data_dir):
    img_path_list = []
    label_list = []
    
    image_path = data_dir
    
#     for product in os.listdir(image_path):

    # get image path
    img_path_list.extend(glob(os.path.join(image_path, '*.jpg')))
    img_path_list.extend(glob(os.path.join(image_path, '*.png')))
    label_list = [ip[len(Test_path):].split('_')[0][1:] for ip in img_path_list]
    # get label
#     label_list.append(''.join(label))
                
    return img_path_list, label_list

In [17]:
test_img_path, test_label_list = get_test_data(Test_path)

In [18]:
print(len(test_label_list))

82


In [19]:
# test data
test_x = test_img_path
# 레이블을 one-hot-vector로 변환
test_y = [label_encoder[key] for key in test_label_list]
test_targets = torch.as_tensor(test_y)
one_hot_test_y = F.one_hot(test_targets)

In [20]:
print([f'{i}, {y}' for i, y in zip(test_label_list, test_y)])

['10097, 1', '10097, 1', '10213, 3', '10213, 3', '10214, 4', '10214, 4', '10215, 5', '10215, 5', '10215, 5', '10266, 6', '10320, 7', '10320, 7', '10321, 8', '15258, 9', '15258, 9', '15458, 10', '15664, 11', '15664, 11', '15804, 12', '15804, 12', '15804, 12', '15896, 13', '15896, 13', '20260, 15', '20260, 15', '20260, 15', '30081, 16', '30163, 17', '30163, 17', '30163, 17', '30164, 18', '30164, 18', '30181, 19', '30181, 19', '30289, 20', '30289, 20', '40182, 21', '40191, 22', '40303, 23', '45707, 24', '50139, 25', '55734, 26', '55734, 26', '55734, 26', '66380, 27', '66380, 27', '66380, 27', '70162, 28', '70162, 28', '70162, 28', '70163, 29', '70180, 30', '70180, 30', '70181, 31', '70181, 31', '70182, 32', '70182, 32', '80140, 33', '80140, 33', '80140, 33', '80146, 34', '80146, 34', '80147, 35', '80147, 35', '80249, 36', '80249, 36', '80250, 37', '80250, 37', '80250, 37', '90174, 38', '90174, 38', '90174, 38', '90176, 39', '90176, 39', '90176, 39', '90245, 40', '90245, 40', '90245, 40', 

# Model

In [21]:
class AlbumentationsCustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True, transforms=None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        # Get image data
        image = cv2.imread(img_path)
        
        # By default OpenCV uses BGR color space for color images,
        # so we need to convert the image to RGB color space.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.train_mode:
#             image = image.astype(np.int16)
            augmented = self.transforms(image=image)
            image = augmented['image']
            label = self.label_list[index]
            return image, label
        else:
            augmented = self.transforms(image=image)
            image = augmented['image']
            label = self.label_list[index]
            return image, label
    
    def __len__(self):
        return len(self.img_path_list)
    
A_test_transform = albumentations.Compose([
                                    A.Resize(256, 256),
#                                     A.Normalize(mean=(0.744859, 0.735139, 0.711357), std=(0.100712, 0.120692, 0.167998)),  # dessert
                                    A.Normalize(mean=(0.719834, 0.727319, 0.72394), std=(0.135879, 0.130642, 0.137573)),   # quasi_drug
#                                     A.pytorch.transforms.ToTensor(),
                                    A.pytorch.transforms.ToTensorV2(transpose_mask=True),
                                ])

# A_vali_dataset = AlbumentationsCustomDataset(x_valid, one_hot_valid_y, train_mode=True, transforms=A_test_transform)
# A_vali_loader = DataLoader(A_vali_dataset, batch_size = 5, shuffle=False, num_workers=0, collate_fn=None)

A_test_dataset = AlbumentationsCustomDataset(test_x, one_hot_test_y, train_mode=False, transforms=A_test_transform)
A_test_loader = DataLoader(A_test_dataset, batch_size = 4, shuffle=False, num_workers=0, collate_fn=None)

In [27]:
class ResNet50(torch.nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()
        model = models.resnet50(pretrained=True)
        modules = list(model.children())[:-1]
        self.feature_extract = nn.Sequential(*modules)
        self.fc1 = nn.Linear(2048, 1000)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(1000,CFG['class'])
#         self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.feature_extract(x)
        # x = x.mean(dim=(-2, -1))
        # (batch, 2048, 4, 4)
        x = torch.squeeze(x)
        x = self.relu(self.fc1(x))
        out = self.fc2(x)
#         out = self.softmax(x)
        
        return out

In [28]:
class EfficientNetb4(torch.nn.Module):
    def __init__(self):
        super(EfficientNetb4, self).__init__()
        model = models.efficientnet_b4(pretrained=True)
        modules = list(model.children())[:-1]
        self.feature_extract = nn.Sequential(*modules)
        self.fc1 = nn.Linear(1792, 1000)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(1000, CFG['class'])
#         self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.feature_extract(x)
        # (batch, 1792, 1, 1)
        x = torch.squeeze(x)
        x = self.relu(self.fc1(x))
        out = self.fc2(x)
#         out = self.softmax(x)
        
        return out

In [29]:
class RegNet(torch.nn.Module):
    def __init__(self):
        super(RegNet, self).__init__()
        model = models.regnet_y_16gf(pretrained=True)
        modules = list(model.children())[:-1]
        self.feature_extract = nn.Sequential(*modules)
        self.fc1 = nn.Linear(3024, 1000)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(1000, CFG['class'])
#         self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.feature_extract(x)
        # (batch, 3024, 1, 1)
        x = torch.squeeze(x)
        x = self.relu(self.fc1(x))
        out = self.fc2(x)
#         out = self.softmax(x)
        
        return out

In [30]:
def score_function(real, pred):
    score = accuracy_score(real, pred)
    return score

# Inference2

In [ ]:
# # sorted(os.listdir('/home/lab17/jupyter_home/saved_models/'))
# file_list = sorted(glob('/home/lab17/jupyter_home/saved_models/*.pth'))
# print(len(file_list))
# sorted(glob('/home/lab17/jupyter_home/saved_models/*.pth'))

In [ ]:
# file_list=['/home/lab17/jupyter_home/saved_models/RegNet_0.001_rmsprop_CosineAnnealing_example.pth']

In [39]:
model_dir = './data/quasi_drug/'

In [40]:
os.listdir(model_dir)

['RegNet_0.0001_rmsprop_CosineAnnealing_example.pth',
 'RegNet_0.001_rmsprop_CosineAnnealing_example.pth',
 'RegNet_1e-05_rmsprop_CosineAnnealing_example.pth']

In [41]:
file_list = sorted(glob(os.path.join(model_dir,'*.pth') ))
file_list

['./data/quasi_drug\\RegNet_0.0001_rmsprop_CosineAnnealing_example.pth',
 './data/quasi_drug\\RegNet_0.001_rmsprop_CosineAnnealing_example.pth',
 './data/quasi_drug\\RegNet_1e-05_rmsprop_CosineAnnealing_example.pth']

In [42]:
pred_ensemble = []
batch_size = 34

device = torch.device('cpu')

for m_path in file_list:
    if 'ResNet50' in m_path:
        model_test = ResNet50().to(device)
    elif 'EfficientNetb4' in m_path:
        model_test = EfficientNetb4().to(device)
    elif 'RegNet' in m_path:
        model_test = RegNet().to(device)    

    model_test.load_state_dict(torch.load(m_path, map_location=device))
    model_test.eval()

    criterion = torch.nn.CrossEntropyLoss()

    test_loss = []
    f_pred = []

#     for img, label in tqdm(iter(A_vali_loader)):
#         img, label = img.float().to(device), label.float().to(device)

#         # Data -> Model -> Output
#         logit = model_test(img)
#         logit = torch.squeeze(logit)

#         # Calc loss
#         loss = criterion(logit, label)

#         test_loss.append(loss.item())
#         f_pred.extend(logit.argmax(1).detach().cpu().numpy().tolist())

#     print('val loss :' ,np.mean(test_loss))
#     print('val acc :', score_function(targets_y, f_pred))

#     test_loss = []
#     f_pred = []

    for img, label in tqdm(iter(A_test_loader)):
        img, label = img.float().to(device), label.float().to(device)

        # Data -> Model -> Output
        logit = model_test(img)
        logit = torch.squeeze(logit)

        # Calc loss
        loss = criterion(logit, label)

        test_loss.append(loss.item())
        f_pred.extend(logit.argmax(1).detach().cpu().numpy().tolist())
        
        
    print(m_path[len(model_dir)+2:])

    print('test loss :' ,np.mean(test_loss))
    print('test acc :', score_function(test_y, f_pred))

  0%|          | 0/21 [00:00<?, ?it/s]

gNet_0.0001_rmsprop_CosineAnnealing_example.pth
test loss : 2.284422179417951
test acc : 0.4878048780487805


  0%|          | 0/21 [00:00<?, ?it/s]

gNet_0.001_rmsprop_CosineAnnealing_example.pth
test loss : 7.407537414914086
test acc : 0.14634146341463414


  0%|          | 0/21 [00:00<?, ?it/s]

gNet_1e-05_rmsprop_CosineAnnealing_example.pth
test loss : 1.7821240786108232
test acc : 0.6463414634146342


In [65]:
def print_ox(test_y,f_pred):
    return 'o' if test_y==f_pred else 'x'
def print_name(test_y, f_pred):
    return f'{label_decoder[y]}, {label_name_decoder[label_decoder[y]]} : {label_name_decoder[label_decoder[p]]}' if test_y!=f_pred else ''
print('o/x | onehot test, pred num | img_num : pred_img_name\n')
for y, p in zip(test_y, f_pred):
    print(f'{print_ox(y,p)} | {y}, {p} | {print_name(y,p)}')
    
    
# print(f'{print_ox(y,p)} | {y}, {p} | {label_decoder[y]} : {label_name_decoder[label_decoder[p]]}')    
#     print(f'{print_ox(y,p)} {y} : {p} {label_name_decoder[y]}, {label_name_decoder[p]}')
# print(f'{test_y}, {f_pred}')
# print(f'{print_ox(test_y,f_pred)} {test_y} : {f_pred} {label_name_decoder[test_y]}, {label_name_decoder[f_pred]}')

o/x | onehot test, pred num | img_num : pred_img_name

x | 1, 27 | 10097, 동아제약)가그린후레쉬라임100ML : 광동제약위생천75ML
x | 1, 7 | 10097, 동아제약)가그린후레쉬라임100ML : 동아제약)가그린어린이용사과
x | 3, 39 | 10213, 클링스스케일링케어스트롱민트치약100G : 유한)좋은느낌대형14맥시슬림
x | 3, 11 | 10213, 클링스스케일링케어스트롱민트치약100G : 고려은단쏠라C구미복숭아맛50G
x | 4, 16 | 10214, 클링스스케일링케어쿨링민트치약100G : 아모레퍼시픽)메디안치석오리지널치약120G
x | 4, 16 | 10214, 클링스스케일링케어쿨링민트치약100G : 아모레퍼시픽)메디안치석오리지널치약120G
o | 5, 5 | 
o | 5, 5 | 
o | 5, 5 | 
o | 6, 6 | 
o | 7, 7 | 
o | 7, 7 | 
x | 8, 24 | 10321, 동국제약)마데카솔 : 유한양행)안티푸라민연고
o | 9, 9 | 
o | 9, 9 | 
o | 10, 10 | 
o | 11, 11 | 
o | 11, 11 | 
o | 12, 12 | 
o | 12, 12 | 
x | 12, 6 | 15804, 동국제약덴트릭스150G : 유한해피홈멸균밴드표준형
o | 13, 13 | 
o | 13, 13 | 
o | 15, 15 | 
o | 15, 15 | 
o | 15, 15 | 
x | 16, 14 | 30081, 아모레퍼시픽)메디안치석오리지널치약120G : 메디안치석케어93_120G
o | 17, 17 | 
x | 17, 23 | 30163, 페리오토탈7스트롱어드밴스치약140G : 애경)2080진지발리스K허벌민트
x | 17, 23 | 30163, 페리오토탈7스트롱어드밴스치약140G : 애경)2080진지발리스K허벌민트
x | 18, 28 | 30164, 부광약품시린메드에프치약 : 부광안티프라그치약
o | 18, 18 | 
x | 19, 39 | 3